# 5. Engenharia de Features - Ames Housing Dataset

Este notebook foca na criação e transformação de features, incluindo:
1. Criação de novas features
2. Transformação de variáveis existentes
3. Codificação de variáveis categóricas
4. Normalização/Padronização de features

In [ ]:
# Importando bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações de visualização
plt.style.use('seaborn')
%matplotlib inline

In [ ]:
# Carregando os dados tratados
df = pd.read_csv('../data/ames_housing_treated.csv')

## 1. Criação de Novas Features

In [ ]:
# Área total
df['TotalArea'] = df['GrLivArea'] + df['TotalBsmtSF']

# Idade da casa no momento da venda
df['HouseAge'] = df['YrSold'] - df['YearBuilt']

# Anos desde a última reforma
df['YearsSinceRemodel'] = df['YrSold'] - df['YearRemodAdd']

# Área total de banheiros
df['TotalBathrooms'] = df['FullBath'] + (0.5 * df['HalfBath']) + \
                       df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath'])

# Área total de pórticos
df['TotalPorchArea'] = df['OpenPorchSF'] + df['EnclosedPorch'] + \
                       df['3SsnPorch'] + df['ScreenPorch']

# Qualidade geral (média das qualidades)
quality_cols = [col for col in df.columns if 'Qual' in col]
df['OverallQuality'] = df[quality_cols].mean(axis=1)

print("Novas features criadas:")
print(df[['TotalArea', 'HouseAge', 'YearsSinceRemodel', 'TotalBathrooms', 
          'TotalPorchArea', 'OverallQuality']].describe())

## 2. Transformação de Variáveis

In [ ]:
# Aplicando log transformation em variáveis numéricas com distribuição assimétrica
skewed_features = ['LotArea', 'TotalArea', 'SalePrice']

for feature in skewed_features:
    df[f'{feature}_Log'] = np.log1p(df[feature])

# Visualizando as distribuições antes e depois da transformação
fig, axes = plt.subplots(len(skewed_features), 2, figsize=(12, 4*len(skewed_features)))

for idx, feature in enumerate(skewed_features):
    # Original
    sns.histplot(df[feature], kde=True, ax=axes[idx, 0])
    axes[idx, 0].set_title(f'Distribuição Original - {feature}')
    
    # Transformada
    sns.histplot(df[f'{feature}_Log'], kde=True, ax=axes[idx, 1])
    axes[idx, 1].set_title(f'Distribuição Log - {feature}')

plt.tight_layout()
plt.show()

## 3. Codificação de Variáveis Categóricas

In [ ]:
# Identificando variáveis categóricas
categorical_cols = df.select_dtypes(include=['object']).columns

# Label Encoding para variáveis ordinais
ordinal_features = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 
                    'HeatingQC', 'KitchenQual', 'FireplaceQu']

for feature in ordinal_features:
    if feature in df.columns:
        le = LabelEncoder()
        df[f'{feature}_Encoded'] = le.fit_transform(df[feature].fillna('None'))

# One-hot encoding para variáveis nominais
nominal_features = [col for col in categorical_cols if col not in ordinal_features]
df_encoded = pd.get_dummies(df, columns=nominal_features, drop_first=True)

print("Dimensões do dataset após codificação:", df_encoded.shape)
print("\nPrimeiras colunas codificadas:")
print(df_encoded.columns[:10])

## 4. Normalização de Features

In [ ]:
# Selecionando features numéricas para normalização
numeric_features = df_encoded.select_dtypes(include=[np.number]).columns
numeric_features = numeric_features.drop('SalePrice')  # Removendo a variável alvo

# Aplicando StandardScaler
scaler = StandardScaler()
df_encoded[numeric_features] = scaler.fit_transform(df_encoded[numeric_features])

# Visualizando distribuição das features normalizadas
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_encoded[numeric_features[:10]])
plt.xticks(rotation=45)
plt.title('Distribuição das Features Normalizadas')
plt.show()

## 5. Salvando o Dataset Processado

In [ ]:
# Salvando o dataset processado
df_encoded.to_csv('../data/ames_housing_processed.csv', index=False)
print("Dataset processado salvo com sucesso!")

# Exibindo informações finais
print(f"\nDimensões finais do dataset: {df_encoded.shape}")
print(f"Número de features numéricas: {len(numeric_features)}")
print(f"Número de features categóricas codificadas: {len(df_encoded.columns) - len(numeric_features) - 1}")